In [ ]:
import sys
import pandas as pd
import datetime as dt
import os

username = os.getlogin()
hoy = dt.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
date_formatcartola = "%d/%m/%Y"
fecha_actual = hoy.strftime(date_format)
fecha_cartola = hoy.strftime(date_formatexport)
df1 = pd.DataFrame()
df2 = pd.DataFrame()
folder_path = 'C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\'
folder_filtros = 'C:\\Users\\' + username + '\\Documents\\Py\\ARCHIVOS PROCESO AM\\FILTROS\\'
filesl = 0
filesh = 0
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
df_list = []
for file in files:
    filesh = len(files)
    df = pd.read_excel(os.path.join(folder_path, file),sheet_name='Movimientos',header=0,names=['fecha', 'sucursal', 'ccte', 'alias', 'ncartola', 'operacion', 'descripcion','cargos','abonos', 'saldo'],thousands='',date_format=date_formatcartola)
    df_list.append(df)
df1 = pd.concat(df_list)
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
df_list = []
for file in files:
    filesl = len(files)
    df = pd.read_csv(os.path.join(folder_path, file),sep=';',header=None,thousands='.',names=['fecha', 'sucursal', 'operacion', 'descripcion', 'cargos', 'abonos', 'saldo'],date_format=date_formatexport)
    df_list.append(df)
df2 = pd.concat(df_list)
if filesh != 1 | filesl != 1: sys.exit()
flineadf = pd.read_table(os.path.join(folder_filtros, 'filtroslinea.txt'),header=None,names=['registros'])
flinea = flineadf['registros'].astype(str).tolist()
fhistdf = pd.read_table(os.path.join(folder_filtros, 'filtroshistorica.txt'),header=None,names=['registros'])
fhist = fhistdf['registros'].astype(str).tolist()
fb2bdf= pd.read_table(os.path.join(folder_filtros, 'filtrosb2b.txt'),header=None,names=['registros'])
fb2b = fb2bdf['registros'].astype(str).tolist()
ffintocdf= pd.read_table(os.path.join(folder_filtros, 'filtrosfintoc.txt'),header=None,names=['registros'])
ffintoc = ffintocdf['registros'].astype(str).tolist()
df1[['operacion','abonos','cargos']] = df1[['operacion','abonos','cargos']].astype(int)
df2[['operacion','abonos','cargos']] = df2[['operacion','abonos','cargos']].astype(int)
df1 = df1.drop(columns=['ccte', 'alias', 'ncartola', 'saldo'])
df2 = df2.drop(columns=['saldo'])
cargos = df1[df1['abonos'] == 0].sort_values(by='descripcion')
b2b = df1[df1['descripcion'].isin(fb2b)].sort_values(by='descripcion')
fintoc = df1[df1['descripcion'].isin(ffintoc)]
rechazo = df1[df1['descripcion'].str.startswith('Rechazo')]
tes = pd.concat([cargos, rechazo])
df1 = df1[~df1['descripcion'].isin(fhist)]
df1 = df1[~df1['descripcion'].str.contains('Rechazo')]
df1 = df1[~df1['descripcion'].str.startswith('Provision en linea')]
df1 = df1[df1['abonos'] != 0]
df2 = df2[~df2['abonos'].isin(fintoc['abonos'])]
df2 = df2[~df2['descripcion'].isin(flinea)]
df2 = df2[~df2['descripcion'].str.contains('Rechazo')]
df2 = df2[df2['abonos'] != 0]
df1['ind'] = 'H'
df2['ind'] = 'L'
dfc = pd.merge(df2, df1, on=['abonos', 'cargos', 'ind'], how='outer')
dfc['fecha_x'] = pd.to_datetime(dfc['fecha_x'], format='mixed', dayfirst=True)
dfc['fecha_y'] = pd.to_datetime(dfc['fecha_y'], format='mixed', dayfirst=True)
dfc.sort_values(by=['abonos', 'fecha_x', 'fecha_y', 'ind'], ascending=[True, True, True, False], inplace=True)
dfc['fecha_x'] = dfc['fecha_x'].dt.strftime(date_formatexport)
dfc['fecha_y'] = dfc['fecha_y'].dt.strftime(date_formatexport)

In [26]:
dfcnar = dfc.copy()
dfcna = dfc.copy()
for value in dfcna['abonos']:
    counth = dfcna[dfcna['ind'].str.contains("H") & (dfcna['abonos'] == value)].shape[0]
    countl = dfcna[dfcna['ind'].str.contains("L") & (dfcna['abonos'] == value)].shape[0]
    if countl > counth:
        dfcna.drop(dfcna[dfcna['abonos'] == value].index[:counth + countl], inplace=True)
        df2.drop(df2[df2['abonos'] == value].index[:countl], inplace=True)
    else:
        dfcna.drop(dfcna[dfcna['abonos'] == value].index[:countl * 2], inplace=True)
        dfcnar.drop(dfcnar[dfcnar['abonos'] == value].index[:counth + countl], inplace=True)
dfcna[['sucursal_x','operacion_x','descripcion_x','fecha_x']] = dfcna[['sucursal_y','operacion_y','descripcion_y','fecha_y']]
dfcna.rename(columns={'sucursal_x': 'sucursal'}, inplace=True)
dfcna.rename(columns={'operacion_x': 'operacion'}, inplace=True)
dfcna.rename(columns={'descripcion_x': 'descripcion'}, inplace=True)
dfcna.rename(columns={'fecha_x': 'fecha'}, inplace=True)
dfcna = dfcna.drop(columns=['sucursal_y', 'operacion_y', 'descripcion_y', 'fecha_y'])
dfcna = dfcna[~dfcna['abonos'].isin(b2b['abonos'])]
dfcna = dfcna[~dfcna['abonos'].isin(fintoc['abonos'])]
dfcna = dfcna[~dfcna['abonos'].isin(cargos['cargos'])]
dfcna = dfcna[~dfcna['abonos'].isin(rechazo['abonos'])]
dfcna = dfcna[dfcna['abonos'] != 0]
cartola = pd.concat([df2, dfcna]).drop(columns=['ind'])